In [21]:
import pandas as pd
import numpy as np
from sklearn.model_selection import LeaveOneOut
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [22]:
df_regions = pd.read_csv('D:/Study-3rd/DSP/OP-heist-project/data/Indices/regional_indexes.csv',encoding = 'iso-8859-1')
df_regions['Regional Import-Export Imbalance'] = df_regions['Regional Import-Export Imbalance']
df_regions['Regional Export Growth'] = df_regions['Regional Export Growth']
df_regions['Regional Import Growth'] = df_regions['Regional Import Growth']
df_regions

,Unnamed: 0,Time,Area,Dominant Industry,Regional export,Regional import,Regional Trade Dependency,Regional Import-Export Imbalance,Regional Export Growth,Regional Import Growth,Regional Export Growth (Absolute),Regional Import Growth (Absolute),Export concentration
0,0,2015,MK01,G,1.691202e+10,3.347404e+10,61.084771,32.870242,0.000000,0.000000,0.000000e+00,0.000000e+00,13.346160
1,1,2015,MK02,Q,4.646912e+09,3.430935e+09,49.373111,-15.053231,0.000000,0.000000,0.000000e+00,0.000000e+00,16.991052
2,2,2015,MK04,C,3.348066e+09,1.207024e+09,55.997258,-47.003293,0.000000,0.000000,0.000000e+00,0.000000e+00,19.388457
3,3,2015,MK05,C,1.288965e+09,7.219806e+08,36.319109,-28.194928,0.000000,0.000000,0.000000e+00,0.000000e+00,16.681200
4,4,2015,MK06,C,4.161019e+09,3.159014e+09,41.505377,-13.688525,0.000000,0.000000,0.000000e+00,0.000000e+00,17.276955
...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,109,2020,MK16,Q,1.892938e+09,2.028934e+09,148.775192,3.467619,10.375421,12.593454,1.779385e+08,2.269340e+08,19.992198
110,110,2020,MK17,Q,1.620991e+09,1.856990e+09,22.960239,6.785509,-28.966215,-15.205952,-6.610090e+08,-3.330103e+08,18.946149
111,111,2020,MK18,Q,3.950119e+08,1.210036e+08,20.525997,-53.100775,-10.224566,23.473111,-4.498809e+07,2.300365e+07,18.777516
112,112,2020,MK19,Q,3.445914e+09,1.600960e+09,73.038080,-36.556370,-9.747799,-18.776227,-3.721801e+08,-3.700885e+08,19.179797


In [23]:
regions = df_regions['Area'].unique().tolist()
years = np.unique(df_regions['Time'])
future_years = [2021, 2022, 2023]
time = [2015,2016,2017,2018,2019,2020,2021,2022,2023]

In [24]:
# Export growths by region
all_export_growth_preds = []
all_exports = []
for region in regions:
        condition1 = df_regions['Area'] == region
        filtered_df = df_regions[condition1]
        if filtered_df.shape[0] == 0:
            continue
        #LOOCV
        loo = LeaveOneOut()
        best_model = None
        best_mse = np.inf  
        for train_index, test_index in loo.split(filtered_df):
            train_data = filtered_df.iloc[train_index]
            test_data = filtered_df.iloc[test_index]
            # Features
            X_train = train_data[['Time']]
            y_train = train_data['Regional Export Growth']
            export_growth_model = LinearRegression()
            export_growth_model.fit(X_train, y_train)
            # Predictions for test set
            export_growth_pred = export_growth_model.predict(test_data[['Time']])
            # Mean squared error for LOOCV
            mse = mean_squared_error(test_data['Regional Export Growth'], export_growth_pred)
            # Identify best model with LOOCV
            if mse < best_mse:
                best_model = export_growth_model
                best_mse = mse
        # Use the best model to make predictions for 3 years in the future
        future_preds_export_growth = []
        for year in future_years:
            X_future = pd.DataFrame({'Time': [year]})
            export_growth_pred = best_model.predict(X_future)
            future_preds_export_growth.append(export_growth_pred[0])
        # Saving results
        all_exports.append(filtered_df['Regional Export Growth'].tolist() + future_preds_export_growth)
# Saving results to csv
df_exports = pd.DataFrame()
df_exports['Time'] = time
index = 0
for region in regions:
        df_exports[region] = all_exports[index]
        index += 1
df_exports.to_csv('D:/Study-3rd/DSP/OP-heist-project/data/forecast_values/exports_growth.csv')

In [25]:
# Do the same for import growths
all_import_growth_preds = []
all_imports = []
for region in regions:
        condition1 = df_regions['Area'] == region
        filtered_df = df_regions[condition1]
        if filtered_df.shape[0] == 0:
            continue
        loo = LeaveOneOut()
        best_model = None
        best_mse = np.inf  
        for train_index, test_index in loo.split(filtered_df):
            train_data = filtered_df.iloc[train_index]
            test_data = filtered_df.iloc[test_index]
            X_train = train_data[['Time']]
            y_train = train_data['Regional Import Growth']
            import_growth_model = LinearRegression()
            import_growth_model.fit(X_train, y_train)
            import_growth_pred = import_growth_model.predict(test_data[['Time']])
            mse = mean_squared_error(test_data['Regional Import Growth'], import_growth_pred)
            if mse < best_mse:
                best_model = import_growth_model
                best_mse = mse
        future_preds_import_growth = []
        for year in future_years:
            X_future = pd.DataFrame({'Time': [year]})
            import_growth_pred = best_model.predict(X_future)
            future_preds_import_growth.append(import_growth_pred[0])
        all_imports.append(filtered_df['Regional Import Growth'].tolist() + future_preds_import_growth)
df_imports = pd.DataFrame()
df_imports['Time'] = time
index = 0
for region in regions:
        df_imports[region] = all_imports[index]
        index += 1
df_imports.to_csv('D:/Study-3rd/DSP/OP-heist-project/data/forecast_values/imports_growth.csv')

In [26]:
# Do the same for trade dependencies
all_trade_growth_preds = []
all_trades = []
for region in regions:
        condition1 = df_regions['Area'] == region
        filtered_df = df_regions[condition1]
        if filtered_df.shape[0] == 0:
            continue
        loo = LeaveOneOut()
        best_model = None
        best_mse = np.inf  
        for train_index, test_index in loo.split(filtered_df):
            train_data = filtered_df.iloc[train_index]
            test_data = filtered_df.iloc[test_index]
            X_train = train_data[['Time']]
            y_train = train_data['Regional Trade Dependency']
            trade_growth_model = LinearRegression()
            trade_growth_model.fit(X_train, y_train)
            trade_growth_pred = trade_growth_model.predict(test_data[['Time']])
            mse = mean_squared_error(test_data['Regional Trade Dependency'], trade_growth_pred)
            if mse < best_mse:
                best_model = trade_growth_model
                best_mse = mse
        future_preds_trade_growth = []
        for year in future_years:
            X_future = pd.DataFrame({'Time': [year]})
            trade_growth_pred = best_model.predict(X_future)
            future_preds_trade_growth.append(trade_growth_pred[0])
        all_trades.append(filtered_df['Regional Trade Dependency'].tolist() + future_preds_trade_growth)
df_trades = pd.DataFrame()
df_trades['Time'] = time
index = 0
for region in regions:
        df_trades[region] = all_trades[index]
        index += 1
df_trades.to_csv('D:/Study-3rd/DSP/OP-heist-project/data/forecast_values/trade_dependencies.csv')

In [27]:
# Do the same for import-export imbalance
all_imba_growth_preds = []
all_imbas = []
for region in regions:
        condition1 = df_regions['Area'] == region
        filtered_df = df_regions[condition1]
        if filtered_df.shape[0] == 0:
            continue
        loo = LeaveOneOut()
        best_model = None
        best_mse = np.inf  
        for train_index, test_index in loo.split(filtered_df):
            train_data = filtered_df.iloc[train_index]
            test_data = filtered_df.iloc[test_index]
            X_train = train_data[['Time']]
            y_train = train_data['Regional Import-Export Imbalance']
            imba_growth_model = LinearRegression()
            imba_growth_model.fit(X_train, y_train)
            imba_growth_pred = imba_growth_model.predict(test_data[['Time']])
            mse = mean_squared_error(test_data['Regional Import-Export Imbalance'], imba_growth_pred)
            if mse < best_mse:
                best_model = imba_growth_model
                best_mse = mse
        future_preds_imba_growth = []
        for year in future_years:
            X_future = pd.DataFrame({'Time': [year]})
            imba_growth_pred = best_model.predict(X_future)
            future_preds_imba_growth.append(imba_growth_pred[0])
        all_imbas.append(filtered_df['Regional Import-Export Imbalance'].tolist() + future_preds_imba_growth)
df_imbas = pd.DataFrame()
df_imbas['Time'] = time
index = 0
for region in regions:
        df_imbas[region] = all_imbas[index]
        index += 1
df_imbas.to_csv('D:/Study-3rd/DSP/OP-heist-project/data/forecast_values/imbalances.csv')

In [28]:
# Exports (Abs) by industry
all_export_growth_abs_preds = []
all_exports_abs = []
for region in regions:
        condition1 = df_regions['Area'] == region
        filtered_df = df_regions[condition1]
        if filtered_df.shape[0] == 0:
            continue
        #LOOCV
        loo = LeaveOneOut()
        best_model = None
        best_mse = np.inf  
        for train_index, test_index in loo.split(filtered_df):
            train_data = filtered_df.iloc[train_index]
            test_data = filtered_df.iloc[test_index]
            # Features
            X_train = train_data[['Time']]
            y_train = train_data['Regional Export Growth (Absolute)']
            export_growth_abs_model = LinearRegression()
            export_growth_abs_model.fit(X_train, y_train)
            # Predictions for test set
            export_growth_abs_pred = export_growth_abs_model.predict(test_data[['Time']])
            # Mean squared error for LOOCV
            mse = mean_squared_error(test_data['Regional Export Growth (Absolute)'], export_growth_abs_pred)
            # Identify best model with LOOCV
            if mse < best_mse:
                best_model = export_growth_abs_model
                best_mse = mse
        # Use the best model to make predictions for 3 years in the future
        future_preds_export_growth_abs = []
        for year in future_years:
            X_future = pd.DataFrame({'Time': [year]})
            export_growth_abs_pred = best_model.predict(X_future)
            future_preds_export_growth_abs.append(export_growth_abs_pred[0])
        # Saving results
        all_exports_abs.append(filtered_df['Regional Export Growth (Absolute)'].tolist() + future_preds_export_growth_abs)
# Saving results to csv
df_exports_abs = pd.DataFrame()
df_exports_abs['Time'] = time
index = 0
for region in regions:
        df_exports_abs[region] = all_exports_abs[index]
        index += 1
df_exports_abs.to_csv('D:/Study-3rd/DSP/OP-heist-project/data/forecast_values/exports_growth_abs.csv')

In [29]:
# Imports (Abs) by industry
all_import_growth_abs_preds = []
all_imports_abs = []
for region in regions:
        condition1 = df_regions['Area'] == region
        filtered_df = df_regions[condition1]
        if filtered_df.shape[0] == 0:
            continue
        #LOOCV
        loo = LeaveOneOut()
        best_model = None
        best_mse = np.inf  
        for train_index, test_index in loo.split(filtered_df):
            train_data = filtered_df.iloc[train_index]
            test_data = filtered_df.iloc[test_index]
            # Features
            X_train = train_data[['Time']]
            y_train = train_data['Regional Import Growth (Absolute)']
            import_growth_abs_model = LinearRegression()
            import_growth_abs_model.fit(X_train, y_train)
            # Predictions for test set
            import_growth_abs_pred = import_growth_abs_model.predict(test_data[['Time']])
            # Mean squared error for LOOCV
            mse = mean_squared_error(test_data['Regional Import Growth (Absolute)'], import_growth_abs_pred)
            # Identify best model with LOOCV
            if mse < best_mse:
                best_model = import_growth_abs_model
                best_mse = mse
        # Use the best model to make predictions for 3 years in the future
        future_preds_import_growth_abs = []
        for year in future_years:
            X_future = pd.DataFrame({'Time': [year]})
            import_growth_abs_pred = best_model.predict(X_future)
            future_preds_import_growth_abs.append(import_growth_abs_pred[0])
        # Saving results
        all_imports_abs.append(filtered_df['Regional Import Growth (Absolute)'].tolist() + future_preds_import_growth_abs)
# Saving results to csv
df_imports_abs = pd.DataFrame()
df_imports_abs['Time'] = time
index = 0
for region in regions:
        df_imports_abs[region] = all_imports_abs[index]
        index += 1
df_imports_abs.to_csv('D:/Study-3rd/DSP/OP-heist-project/data/forecast_values/imports_growth_abs.csv')

In [30]:
# Exports by industry
all_exp_preds = []
all_exp = []
for region in regions:
        condition1 = df_regions['Area'] == region
        filtered_df = df_regions[condition1]
        if filtered_df.shape[0] == 0:
            continue
        #LOOCV
        loo = LeaveOneOut()
        best_model = None
        best_mse = np.inf  
        for train_index, test_index in loo.split(filtered_df):
            train_data = filtered_df.iloc[train_index]
            test_data = filtered_df.iloc[test_index]
            # Features
            X_train = train_data[['Time']]
            y_train = train_data['Regional export']
            exp_model = LinearRegression()
            exp_model.fit(X_train, y_train)
            # Predictions for test set
            exp_pred = exp_model.predict(test_data[['Time']])
            # Mean squared error for LOOCV
            mse = mean_squared_error(test_data['Regional export'], exp_pred)
            # Identify best model with LOOCV
            if mse < best_mse:
                best_model = exp_model
                best_mse = mse
        # Use the best model to make predictions for 3 years in the future
        future_preds_exp = []
        for year in future_years:
            X_future = pd.DataFrame({'Time': [year]})
            exp_pred = best_model.predict(X_future)
            future_preds_exp.append(exp_pred[0])
        # Saving results
        all_exp.append(filtered_df['Regional export'].tolist() + future_preds_exp)
# Saving results to csv
df_exp = pd.DataFrame()
df_exp['Time'] = time
index = 0
for region in regions:
        df_exp[region] = all_exp[index]
        index += 1
df_exp.to_csv('D:/Study-3rd/DSP/OP-heist-project/data/forecast_values/exports.csv')

In [31]:
# Imports by industry
all_imp_preds = []
all_imp = []
for region in regions:
        condition1 = df_regions['Area'] == region
        filtered_df = df_regions[condition1]
        if filtered_df.shape[0] == 0:
            continue
        #LOOCV
        loo = LeaveOneOut()
        best_model = None
        best_mse = np.inf  
        for train_index, test_index in loo.split(filtered_df):
            train_data = filtered_df.iloc[train_index]
            test_data = filtered_df.iloc[test_index]
            # Features
            X_train = train_data[['Time']]
            y_train = train_data['Regional import']
            imp_model = LinearRegression()
            imp_model.fit(X_train, y_train)
            # Predictions for test set
            imp_pred = imp_model.predict(test_data[['Time']])
            # Mean squared error for LOOCV
            mse = mean_squared_error(test_data['Regional import'], imp_pred)
            # Identify best model with LOOCV
            if mse < best_mse:
                best_model = imp_model
                best_mse = mse
        # Use the best model to make predictions for 3 years in the future
        future_preds_imp = []
        for year in future_years:
            X_future = pd.DataFrame({'Time': [year]})
            imp_pred = best_model.predict(X_future)
            future_preds_imp.append(imp_pred[0])
        # Saving results
        all_imp.append(filtered_df['Regional import'].tolist() + future_preds_imp)
# Saving results to csv
df_imp = pd.DataFrame()
df_imp['Time'] = time
index = 0
for region in regions:
        df_imp[region] = all_imp[index]
        index += 1
df_imp.to_csv('D:/Study-3rd/DSP/OP-heist-project/data/forecast_values/imports.csv')